In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
path = 'C:/Users/new/Desktop/train.xlsx'
data = pd.read_excel(path)

In [3]:
data.head()

,Date,township,经度,纬度,经停病例数,患者数,全市患者数,所在区患者数,封控与否
0,2021-12-23,咸水沽镇,117.395242,38.979148,11,0,0,0,0
1,2021-12-24,咸水沽镇,117.395242,38.979148,28,0,0,0,0
2,2021-12-25,咸水沽镇,117.395242,38.979148,28,0,0,0,0
3,2021-12-26,咸水沽镇,117.395242,38.979148,28,0,0,0,0
4,2021-12-27,咸水沽镇,117.395242,38.979148,54,0,0,0,0


In [4]:
data.drop(['township','经度','纬度'], axis=1, inplace=True)
data.set_index('Date',inplace=True)

In [5]:
data.head()

,经停病例数,患者数,全市患者数,所在区患者数,封控与否
Date,,,,,
2021-12-23,11,0,0,0,0
2021-12-24,28,0,0,0,0
2021-12-25,28,0,0,0,0
2021-12-26,28,0,0,0,0
2021-12-27,54,0,0,0,0


In [6]:
data.shape[0]
data.shape[1]

41

5

In [7]:
split = int(len(data)*0.8)
train = data[:split]
test = data[split:]

In [8]:
train.head()

,经停病例数,患者数,全市患者数,所在区患者数,封控与否
Date,,,,,
2021-12-23,11,0,0,0,0
2021-12-24,28,0,0,0,0
2021-12-25,28,0,0,0,0
2021-12-26,28,0,0,0,0
2021-12-27,54,0,0,0,0


In [9]:
test.head()

,经停病例数,患者数,全市患者数,所在区患者数,封控与否
Date,,,,,
2022-01-24,0,62,265,253,1
2022-01-25,0,51,225,217,1
2022-01-26,0,49,226,213,0
2022-01-27,0,37,188,178,0
2022-01-28,0,29,169,154,0


In [10]:
scaler = MinMaxScaler(feature_range=(0,1))
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [11]:
train

array([[0.14285714, 0.        , 0.        , 0.        , 0.        ],
       [0.36363636, 0.        , 0.        , 0.        , 0.        ],
       [0.36363636, 0.        , 0.        , 0.        , 0.        ],
       [0.36363636, 0.        , 0.        , 0.        , 0.        ],
       [0.7012987 , 0.        , 0.        , 0.        , 0.        ],
       [0.63636364, 0.        , 0.        , 0.        , 0.        ],
       [0.64935065, 0.        , 0.        , 0.        , 0.        ],
       [0.71428571, 0.        , 0.        , 0.        , 0.        ],
       [0.71428571, 0.        , 0.        , 0.        , 0.        ],
       [0.62337662, 0.        , 0.        , 0.        , 0.        ],
       [0.61038961, 0.        , 0.        , 0.        , 0.        ],
       [0.50649351, 0.        , 0.        , 0.        , 0.        ],
       [0.83116883, 0.        , 0.        , 0.        , 0.        ],
       [0.80519481, 0.        , 0.        , 0.        , 0.        ],
       [0.76623377, 0.        , 0.

In [12]:
test

array([[0.        , 0.69662921, 0.72207084, 0.72701149, 1.        ],
       [0.        , 0.57303371, 0.61307902, 0.62356322, 1.        ],
       [0.        , 0.5505618 , 0.61580381, 0.61206897, 0.        ],
       [0.        , 0.41573034, 0.51226158, 0.51149425, 0.        ],
       [0.        , 0.3258427 , 0.46049046, 0.44252874, 0.        ],
       [0.        , 0.23595506, 0.32425068, 0.27298851, 0.        ],
       [0.        , 0.15730337, 0.24250681, 0.15517241, 0.        ],
       [0.        , 0.06741573, 0.1852861 , 0.09482759, 0.        ],
       [0.        , 0.05617978, 0.22615804, 0.07758621, 0.        ]])

In [21]:
def createXY(dataset, n_past):
    col = dataset.shape[1]
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
        dataX.append(dataset[i - n_past:i, 0:col-1]) 
        dataY.append(dataset[i,col-1]) # 最后一列是封控与否
    return np.array(dataX),np.array(dataY)

train_X, train_Y=createXY(train, 7)
test_X, test_Y=createXY(test, 7)

In [22]:
train_X

array([[[0.14285714, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.7012987 , 0.        , 0.        , 0.        ],
        [0.63636364, 0.        , 0.        , 0.        ],
        [0.64935065, 0.        , 0.        , 0.        ]],

       [[0.36363636, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.7012987 , 0.        , 0.        , 0.        ],
        [0.63636364, 0.        , 0.        , 0.        ],
        [0.64935065, 0.        , 0.        , 0.        ],
        [0.71428571, 0.        , 0.        , 0.        ]],

       [[0.36363636, 0.        , 0.        , 0.        ],
        [0.36363636, 0.        , 0.        , 0.        ],
        [0.7012987 , 0.        , 0.        , 0.        ],
        [0

In [23]:
train_Y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [37]:
def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(50,return_sequences=True,input_shape=(7,4)))
    grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(1))
    grid_model.compile(loss = 'mse',optimizer = optimizer)
    return grid_model 

grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(test_X,test_Y))



parameters = {'batch_size' : [16,20],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 2)

C:\Users\new\AppData\Local\Temp\ipykernel_52748\385576689.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(test_X,test_Y))


In [38]:
grid_search = grid_search.fit(train_X,train_Y)

Epoch 1/8
1/1 [==============================] - 8s 8s/step - loss: 1.0862 - val_loss: 1.7378e-04
Epoch 2/8
1/1 [==============================] - 0s 45ms/step - loss: 0.9585 - val_loss: 0.0026
Epoch 3/8
1/1 [==============================] - 0s 46ms/step - loss: 0.8959 - val_loss: 0.0081
Epoch 4/8
1/1 [==============================] - 0s 44ms/step - loss: 0.7829 - val_loss: 0.0171
Epoch 5/8
1/1 [==============================] - 0s 44ms/step - loss: 0.7249 - val_loss: 0.0300
Epoch 6/8
1/1 [==============================] - 0s 47ms/step - loss: 0.6126 - val_loss: 0.0477
Epoch 7/8
1/1 [==============================] - 0s 47ms/step - loss: 0.6071 - val_loss: 0.0712
Epoch 8/8
1/1 [==============================] - 0s 26ms/step - loss: 0.2186
Epoch 1/8
1/1 [==============================] - 3s 3s/step - loss: 0.3383 - val_loss: 5.9174e-05
Epoch 2/8
1/1 [==============================] - 0s 45ms/step - loss: 0.3216 - val_loss: 6.5243e-04
Epoch 3/8
1/1 [==============================] - 0s

1/1 [==============================] - 0s 45ms/step - loss: 0.5523 - val_loss: 0.0860
Epoch 8/8
1/1 [==============================] - 0s 27ms/step - loss: 0.2462
Epoch 1/8
1/1 [==============================] - 3s 3s/step - loss: 0.3027 - val_loss: 7.9790e-05
Epoch 2/8
1/1 [==============================] - 0s 45ms/step - loss: 0.2857 - val_loss: 4.7680e-04
Epoch 3/8
1/1 [==============================] - 0s 47ms/step - loss: 0.2664 - val_loss: 0.0013
Epoch 4/8
1/1 [==============================] - 0s 49ms/step - loss: 0.2434 - val_loss: 0.0025
Epoch 5/8
1/1 [==============================] - 0s 46ms/step - loss: 0.2330 - val_loss: 0.0043
Epoch 6/8
1/1 [==============================] - 0s 48ms/step - loss: 0.2234 - val_loss: 0.0068
Epoch 7/8
1/1 [==============================] - 0s 48ms/step - loss: 0.2107 - val_loss: 0.0100
Epoch 8/8
1/1 [==============================] - 0s 27ms/step - loss: 0.7104
Epoch 1/8
1/1 [==============================] - 3s 3s/step - loss: 1.1977 - val_l

In [39]:
grid_search.best_params_

{'batch_size': 16, 'epochs': 10, 'optimizer': 'adam'}

In [40]:
my_model = grid_search.best_estimator_.model

In [41]:
prediction=my_model.predict(test_X)
print("prediction\n", prediction)
print("\nPrediction Shape-",prediction.shape)

1/1 [==============================] - 1s 583ms/step
prediction
 [[0.96959776]
 [0.85554886]]

Prediction Shape- (2, 1)
